In [1]:
import requests
r = requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")
with open('spacex_dash_app.py','wb') as File1:
    File1.write(r.content)

In [3]:
import requests
r = requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
with open('spacex_launch_dash.csv','wb') as File1:
    File1.write(r.content)

### Most of the python code is taken from the downloaded python file above

In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [16]:
spacex_df[spacex_df['Launch Site'] == 'KSC LC-39A']

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
36,36,30,KSC LC-39A,Success,1,2490.00,F9 FT B1031.1,FT
37,37,31,KSC LC-39A,Success,0,5600.00,F9 FT B1030,FT
38,38,32,KSC LC-39A,Success,1,5300.00,F9 FT B1021.2,FT
39,39,33,KSC LC-39A,Success,1,3696.65,F9 FT B1032.1,FT
40,40,34,KSC LC-39A,Success,0,6070.00,F9 FT B1034,FT
41,41,35,KSC LC-39A,Success,1,2708.00,F9 FT B1035.1,FT
42,42,36,KSC LC-39A,Success,1,3669.00,F9 FT B1029.2,FT
43,43,38,KSC LC-39A,Success,0,6761.00,F9 FT B1037,FT
44,44,39,KSC LC-39A,Success,1,3310.00,F9 B4 B1039.1,B4
45,45,41,KSC LC-39A,Success,1,4990.00,F9 B4 B1040.1,B4


In [39]:
max(spacex_df['Payload Mass (kg)'])

9600.0

In [46]:
px.pie(spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'], names='class', 
title='title')

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the spacex data into a pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                  dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                            min=0, max=10000, step=1000,
                                            marks={0: '0',
                                                   100: '100'},
                                            value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.pie(filtered_df[filtered_df['Launch Site'] == str(entered_site)], names='class', 
        title='Total Success Launches for site' + entered_site)
        return fig
            
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload):
    filtered_df_2 = spacex_df[(spacex_df['Payload Mass (kg)'] > payload[0]) & (spacex_df['Payload Mass (kg)'] < payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(data_frame=filtered_df_2, x='Payload Mass (kg)', y='class', 
        color="Booster Version Category", 
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.scatter(filtered_df_2[filtered_df_2['Launch Site'] == str(entered_site)], x='Payload Mass (kg)', y='class', 
        color="Booster Version Category", 
        title='Correlation between Payload and Success for all Sites')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 20:04:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:04:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:04:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:04:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:04:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 20:05:20] "POST /_dash-update-component

In [49]:
fig = px.scatter(data_frame=spacex_df, x='Payload Mass (kg)', y='class', 
color="Booster Version Category", 
title='Correlation between Payload and Success for all Sites')
fig